## Dependencies

In [1]:
import re
import warnings
import pandas as pd
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, Average, BatchNormalization, Activation, Add
from googleqa_utilityscript import *

SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
model_path = '/kaggle/input/51-googleq-a-train-use-skip-add/use.h5'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

## Test set

In [5]:
# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']

X_test = [X_test_title, X_test_body, X_test_answer]

# Model parameters

In [6]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
    return encoder

In [8]:
# Title
input_title = Input(shape=(1,), dtype=tf.string, name='input_title')
encoded_title = encoder_block(input_title)

# Body
input_body = Input(shape=(1,), dtype=tf.string, name='input_body')
encoded_body = encoder_block(input_body)

# Answer
input_answer = Input(shape=(1,), dtype=tf.string, name='input_answer')
encoded_answer = encoder_block(input_answer)


# Output
title = Dense(256, kernel_initializer='glorot_uniform')(encoded_title)
title = BatchNormalization()(title)
title = Activation('relu')(title)
title = Dropout(0.5)(title)
title = Dense(128, kernel_initializer='glorot_uniform')(title)
title = BatchNormalization()(title)
title_out = Activation('relu')(title)

body = Dense(256, kernel_initializer='glorot_uniform')(encoded_body)
body = BatchNormalization()(body)
body = Activation('relu')(body)
body = Dropout(0.5)(body)
body = Dense(128, kernel_initializer='glorot_uniform')(body)
body = BatchNormalization()(body)
body_out = Activation('relu')(body)

answer = Dense(512, kernel_initializer='glorot_uniform')(encoded_answer)
answer = BatchNormalization()(answer)
answer_out = Activation('relu')(answer)

question = Add()([title_out, body_out])
question = Dense(512, kernel_initializer='glorot_uniform')(question)
question = BatchNormalization()(question)
question_out = Activation('relu')(question)

x = Add()([question_out, answer_out, encoded_title, encoded_body, encoded_answer])
x = Dropout(0.5)(x)
x = Dense(512, kernel_initializer='glorot_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Add()([x, encoded_title, encoded_body, encoded_answer])
x = Dense(512, kernel_initializer='glorot_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='glorot_uniform', name='output')(x)
model = Model(inputs=[input_title, input_body, input_answer], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_body (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512)          0           input_title[0][0]                
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512)          0           input_body[0][0]                 
______________________________________________________________________________________________

# Make predictions

In [9]:
Y_test = model.predict(X_test)

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.900779,0.667873,0.095628,0.779224,0.765549,0.826895,0.620211,0.566240,0.348457,...,0.871656,0.925027,0.650493,0.963051,0.962682,0.853720,0.139369,0.085848,0.764545,0.905944
1,46,0.853014,0.525739,0.001569,0.701250,0.797617,0.923606,0.524437,0.415508,0.150875,...,0.696026,0.915851,0.618769,0.949753,0.967684,0.830868,0.918063,0.156809,0.070703,0.861111
2,70,0.888343,0.673763,0.030403,0.785053,0.916364,0.926838,0.598302,0.537266,0.415451,...,0.847308,0.910495,0.653663,0.956255,0.963560,0.856074,0.066419,0.088527,0.794630,0.895530
3,132,0.827516,0.414274,0.004690,0.701238,0.708983,0.917169,0.519032,0.411517,0.155464,...,0.639857,0.929304,0.633381,0.961731,0.975057,0.841346,0.827356,0.154444,0.666508,0.885692
4,200,0.889334,0.511493,0.035904,0.768347,0.848264,0.872080,0.590499,0.511809,0.395517,...,0.781898,0.939157,0.670760,0.973983,0.976040,0.877410,0.186813,0.129750,0.872753,0.919272
